In [25]:
import torch
import torchvision
from torchvision import transforms # 이미지 데이터 transform
from torch.utils.data import DataLoader # 이미지 데이터 로더
import os
import glob
import numpy as np
from PIL import Image

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [27]:
train_path = "/content/drive/MyDrive/socar/accida_segmentation_dataset_v1/dent/train"

img_path = "images"
mask_path = "masks"

In [28]:
dent_img_path = os.path.join(train_path, img_path)
img = np.array(Image.open(dent_img_path).convert('RGB'))


IsADirectoryError: ignored

In [ ]:
resize_trans = transforms.Compose([
                                   transforms.Resize((224, 224)),
                                   transforms.ToTensor()
])

resize_train = torchvision.datasets.ImageFolder(root=train_path, transform=resize_trans)

In [ ]:
resize_train[0][0].shape

torch.Size([3, 224, 224])

In [ ]:
resize_train[0][0]

tensor([[[0.0275, 0.0314, 0.0314,  ..., 0.4824, 0.4824, 0.4824],
         [0.0275, 0.0314, 0.0314,  ..., 0.4902, 0.4863, 0.4863],
         [0.0275, 0.0314, 0.0314,  ..., 0.4980, 0.4941, 0.4902],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.1843, 0.1882, 0.1922,  ..., 0.4549, 0.4549, 0.4549],
         [0.1843, 0.1882, 0.1922,  ..., 0.4627, 0.4588, 0.4588],
         [0.1843, 0.1882, 0.1922,  ..., 0.4706, 0.4667, 0.4627],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.1843, 0.1961, 0.2039,  ..., 0.3412, 0.3412, 0.3412],
         [0.1843, 0.1961, 0.2039,  ..., 0.3490, 0.3451, 0.3451],
         [0.1843, 0.1961, 0.2039,  ..., 0.3569, 0.3529, 0.

In [ ]:
np.mean(resize_train[0][0].numpy(),axis=(1,2)) # numpy로 바꾸고, axis = 1,2 mean으로 RGB mean/std 뽑기

array([0.35136792, 0.35165766, 0.318502  ], dtype=float32)

In [ ]:
def get_mean_std(dataset):
  meanRGB = [np.mean(image.numpy(), axis=(1,2)) for image,_ in dataset]
  stdRGB = [np.std(image.numpy(), axis=(1,2)) for image,_ in dataset]

  meanR = np.mean([m[0] for m in meanRGB])
  meanG = np.mean([m[1] for m in meanRGB])
  meanB = np.mean([m[2] for m in meanRGB])

  stdR = np.mean([s[0] for s in stdRGB])
  stdG = np.mean([s[1] for s in stdRGB])
  stdB = np.mean([s[2] for s in stdRGB])

  print(meanR, meanG, meanB)
  print(stdR, stdG, stdB)

In [ ]:
get_mean_std(resize_train)

0.23833017 0.2355441 0.23113322
0.14454623 0.14360258 0.14423908


In [21]:
# normalization 준비

resize_train_mean=[0.23833017, 0.2355441, 0.231133222]
resize_train_std=[0.14454623, 0.14360258, 0.14423908]

In [24]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)), # 이미지 resize
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2), # 이미지 지터링(밝기, 대조, 채비, 색조)
    transforms.ToTensor(),
    transforms.Normalize(resize_train_mean, resize_train_std)
])


trainset = torchvision.datasets.ImageFolder(root=train_path, transform=transform_train)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=0)



FileNotFoundError: ignored